In [11]:
import os, json, pickle, pprint, langdetect, requests
root_data = "/data/cm/"
root = "/home/rovera/cm/"

In [12]:
def retrieve_journal_index(journal):
    # if the index already exists, open and return it, otherwise return an empty dictionary
    journal_index = {}
    try:
        journal_index = json.load(open(root+"output/linker/indexes/journals/"+journal+'.json', "r", encoding='utf-8'))
    except FileNotFound as fnf:
        pass
    
    return journal_index


def get_pages_to_process(journal):
    pass

    
    
def tag_text(data):
    
    params = {
    'text': data,
    'gcube-token': '082285fb-1487-41fc-a75d-77be6c3a3780-843339462',
    'lang': 'de',
    'epsilon': 0.05
    }
    
    r = requests.post('https://tagme.d4science.org/tagme/tag', data=params)
    
    res = r.json()
    
    return res
    
    
def filter_result(res, link_prob=0.25, rho=0.25):
    return [r for r in res['annotations'] if r['link_probability'] > link_prob and r['rho'] > rho]

def save_journal_index(journal_index, journal_index_name):
    json.dump(journal_index, open(root+"data/linker_indexes/"+journal_index_name+".json", 'w', encoding="utf-8"), ensure_ascii=False, indent=2)

In [13]:
# NOT TO REUSE, IT OVERWRITES!!!
# create a list of journals in order to keep track of which journals have been processed
#proc_journals = {}
#for j in os.listdir(root_data+"output/text/"):
#    proc_journals[j] = False
#with open(root_data+"output/linker/indexes/journals_status.json", "w", encoding="utf-8") as outfile:
#    json.dump(proc_journals, outfile, indent=2)

In [14]:
# NOT TO REUSE, IT OVERWRITES!!!
# first iterate over the journals and collect the pages to be processed
# create
# a) an output folder named according to the journal id that will contain a file with entities for each page
# b) an index for each journal that describes the processing status of each page
"""    
for journal in os.listdir(root_data+"output/text/"):
        
    print("Processing {}...".format(journal), end="")
        
    # create an output folder for the journal
    if not os.path.exists(root_data+"output/linker/entities/"+journal):
        os.makedirs(root_data+"output/linker/entities/"+journal)
        
    # create and index file for that journal
    journal_index = {}
        
    for (rs, ds, fs) in os.walk(root_data+"output/text/"+journal+"/"):
            
        if fs != []:
            pathname = rs.split('text/')[1].replace('/', '_')

            for file in fs:
                curr_filename = file.split('.')[0]
                output_filename = pathname + "__" + curr_filename
                    
                journal_index[output_filename] = False
                    
    with open(root_data+"output/linker/indexes/journals/"+journal+".json", "w", encoding="utf-8") as outfile:
        json.dump(journal_index, outfile, ensure_ascii=False, indent=2)
        
    print("...Done.")
"""

'    \nfor journal in os.listdir(root_data+"output/text/"):\n        \n    print("Processing {}...".format(journal), end="")\n        \n    # create an output folder for the journal\n    if not os.path.exists(root_data+"output/linker/entities/"+journal):\n        os.makedirs(root_data+"output/linker/entities/"+journal)\n        \n    # create and index file for that journal\n    journal_index = {}\n        \n    for (rs, ds, fs) in os.walk(root_data+"output/text/"+journal+"/"):\n            \n        if fs != []:\n            pathname = rs.split(\'text/\')[1].replace(\'/\', \'_\')\n\n            for file in fs:\n                curr_filename = file.split(\'.\')[0]\n                output_filename = pathname + "__" + curr_filename\n                    \n                journal_index[output_filename] = False\n                    \n    with open(root_data+"output/linker/indexes/journals/"+journal+".json", "w", encoding="utf-8") as outfile:\n        json.dump(journal_index, outfile, ensure

In [15]:
journals_status = json.load(open(root_data+"output/linker/indexes/journals_status.json", "r"))
for journal, status in journals_status.items():
        
    if status is False:
        
        print("Processing journal {}...".format(journal))
        
        c = 0
        
        # open corresponding journal index
        journal_index = json.load(open(root_data+"output/linker/indexes/journals/"+journal+".json", "r"))
        
        tot = len(journal_index)
        to_process = len([k for k,v in journal_index.items() if v is False])
        
        print("{} total pages in this journal.".format(str(tot)))
        print("{} pages still to process.".format(str(to_process)))
        
        for page, page_status in journal_index.items():
            
            if page_status is False:
                
                # open corresponding file for processing
                pagepath, pagename = page.split("__")
                pagepath = pagepath.replace('_', '/') + '/'
                
                pagename = pagename.lstrip("_")
                
                data = open(root_data+"output/text/"+pagepath+pagename+".txt", "r", encoding="utf-8").read()
                
                # check language
                try:
                    lang = langdetect.detect(data)
                except langdetect.lang_detect_exception.LangDetectException as lde:
                    lang = None
                    #print(lde)
                    
                # tag the data
                if lang == 'de':
                    res = tag_text(data)
                    res = filter_result(res)
            
                    # save the result
                    json.dump(res, open(root_data+"output/linker/entities/"+journal+"/"+page+".json", "w", encoding="utf-8"), ensure_ascii=False, indent=2)
                    # update the index with status for the current page
                    journal_index[page] = True
            
            
                else: # if data is not in german, only update the index with status for the current page
                    if isinstance(lang, str) and lang != 'de':
                        journal_index[page] = "Not in German"
                    else:
                        journal_index[page] = None
        
                # save back updated journal index
                with open(root_data+"output/linker/indexes/journals/"+journal+".json", "w") as outfile:
                     json.dump(journal_index, outfile, indent=2)
            
            c += 1
            if c % 10 == 0:
                print(c, end=" ")
    
        # update journal status to True
        journals_status[journal] = True
        with open(root_data+"output/linker/indexes/journals_status.json", "w") as o_file:
            json.dump(journals_status, o_file, indent=2)
        print("Done.")
        print()
        
        #ask_user = input("Would you like to proceed to the next journal? (y/n)")
        #ask_user = 'y'
        #if ask_user in ('y', 'yes', 'Y', 'YES', 'jo'):
        #    continue
        #else:
        #    break
    
    #break
    else:
        print("Journal {} has already been processed.".format(journal))

Journal 2710055 has already been processed.
Journal 9616701 has already been processed.
Journal 2580773 has already been processed.
Journal 2895450 has already been processed.
Journal 2727810 has already been processed.
Journal 3129675 has already been processed.
Journal 6492429 has already been processed.
Journal 2438141 has already been processed.
Journal 2827798 has already been processed.
Journal 5438908 has already been processed.
Journal 4782723 has already been processed.
Journal 7506414 has already been processed.
Journal 9054153 has already been processed.
Journal 9366335 has already been processed.
Journal 4889892 has already been processed.
Journal 2425321 has already been processed.
Journal 6492329 has already been processed.
Journal 700673 has already been processed.
Journal 4895413 has already been processed.
Journal 2542917 has already been processed.
Journal 9583346 has already been processed.
Journal 2549643 has already been processed.
Journal 10638461 has already been